<a href="https://colab.research.google.com/github/Urvashi-D/hacktoberfest/blob/urvashi_project1/project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libraries for natural language processing
!pip install spacy
!python -m spacy download en_core_web_sm

# Install SQLite for database querying (optional if you plan to test SQL queries)
import sqlite3


In [13]:
import spacy

nlp = spacy.load('en_core_web_sm')
def break_down_query(query_text):
    parsed_query = nlp(query_text)
    word_list = [word.text for word in parsed_query]
    return word_list
def create_sql_query(query_text):
     if "orders" in query_text and "completed" in query_text and "July" in query_text:
        return "SELECT * FROM orders WHERE status = 'completed' AND month = 'July';"
     else:
        return "I don't understand the query."
user_query = "Get all completed orders in July"
parsed_words = break_down_query(user_query)
sql_statement = create_sql_query(user_query)
print("Words from query:", parsed_words)
print("Generated SQL:", sql_statement)


Words from query: ['Get', 'all', 'completed', 'orders', 'in', 'July']
Generated SQL: SELECT * FROM orders WHERE status = 'completed' AND month = 'July';


In [15]:
import spacy
nlp = spacy.load('en_core_web_sm')
tables = {
    "orders": ["order_id", "customer_id", "status", "month", "amount"],
    "customers": ["customer_id", "name", "email", "city"],
}
def break_down_query(query_text):
    parsed_query = nlp(query_text)
    word_list = [word.text for word in parsed_query]
    return word_list
def table_contains_column(table, column):
    return column in tables[table]
def create_sql_query(query_text):
    tokens = break_down_query(query_text)
    select_part = "SELECT *"
    from_part = ""
    where_part = ""
    join_part = ""
    if "customer" in tokens and "order" in tokens:
        from_part = " FROM orders"
        join_part = " JOIN customers ON orders.customer_id = customers.customer_id"
        if "city" in tokens:
            where_part = " WHERE customers.city = 'Indore'"
    elif "orders" in tokens:
        from_part = " FROM orders"
        if "completed" in tokens and "July" in tokens:
            where_part = " WHERE status = 'completed' AND month = 'July'"
    if "order by" in query_text:
        select_part += " ORDER BY amount DESC"
    sql_query = select_part + from_part + join_part + where_part
    return sql_query
user_query = "Get all customers who ordered from Indore in July"
parsed_words = break_down_query(user_query)
sql_statement = create_sql_query(user_query)
print("Words from query:", parsed_words)
print("Generated SQL:", sql_statement)


Words from query: ['Get', 'all', 'customers', 'who', 'ordered', 'from', 'Indore', 'in', 'July']
Generated SQL: SELECT *


In [ ]:
pip install sqlalchemy sqlite

In [ ]:
from sqlalchemy import create_engine, Column, String, Float, Integer, DateTime, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import pandas as pd
from datetime import datetime
Base = declarative_base()
DATABASE_URL = "sqlite:///orders.db"
engine = create_engine(DATABASE_URL)
class OrderHeader(Base):
    __tablename__ = 'order_header'
    orderId = Column(String, primary_key=True)
    orderDate = Column(DateTime)
    orderStatus = Column(String)
    grandTotal = Column(Float)
    items = relationship("OrderItem", back_populates="order_header")
class OrderItem(Base):
    __tablename__ = 'order_item'
    orderItemSeqId = Column(String, primary_key=True)
    orderId = Column(String, ForeignKey('order_header.orderId'))
    productId = Column(String)
    quantity = Column(Integer)
    unitPrice = Column(Float)
    order_header = relationship("OrderHeader", back_populates="items")
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()
def add_order(order_info):
    """
    Add a new order to the database.

    Args:
        order_info (dict): Information about the order and its items.
    """
    try:
        order_date = datetime.strptime(order_info["orderDate"], "%Y-%m-%d %H:%M:%S")
        new_order = OrderHeader(
            orderId=order_info["orderId"],
            orderDate=order_date,
            orderStatus=order_info["orderStatus"],
            grandTotal=order_info["grandTotal"]
        )
        session.add(new_order)
        for item in order_info["OrderItems"]:
            order_item = OrderItem(
                orderItemSeqId=item["orderItemSeqId"],
                orderId=order_info["orderId"],
                productId=item["productId"],
                quantity=item["quantity"],
                unitPrice=item["unitPrice"]
            )
            session.add(order_item)
        session.commit()
        print("Order has been added successfully!")

    except Exception as e:
        print(f"An error occurred while adding the order: {e}")
        session.rollback()  # Rollback the session on error
sample_order = {
    "orderId": "10001",
    "orderDate": "2020-05-28 08:50:43",
    "orderStatus": "ORDER_CREATED",
    "grandTotal": 95.00,
    "OrderItems": [
        {
            "orderItemSeqId": "00001",
            "productId": "P100",
            "quantity": 2,
            "unitPrice": 20.00
        },
        {
            "orderItemSeqId": "00002",
            "productId": "P200",
            "quantity": 1,
            "unitPrice": 50.00
        }
    ]
}
add_order(sample_order)

def prepare_data_for_ml():

    orders = session.query(OrderHeader).all()
    processed_data = []
    for order in orders:
        total_items_count = len(order.items)
        total_quantity = sum(item.quantity for item in order.items)
        total_amount = order.grandTotal

        processed_data.append({
            "orderId": order.orderId,
            "total_items": total_items_count,
            "total_quantity": total_quantity,
            "grand_total": total_amount
        })
    return pd.DataFrame(processed_data)
order_dataframe = prepare_data_for_ml()
print(order_dataframe)
session.close()
